In [ ]:
import pandas as pd
import sqlite3
import numpy as np
import re
import pymysql
import configparser as cp
import pykml.parser

from datetime import datetime
from pymysql.cursors import DictCursor
from sys import stderr
from os import environ, path, remove as os_remove
from IPython.display import display

In [ ]:
COUNTRIES_FILE = "countries.txt"
GBIF_CSV = "GBIF_bee_occurrences_2021_harvest.csv"
N_AMERICA_KML = "NAmerica.kml"
PROVINCES_FILE = "provinces.txt"
SCAN_SQLITE = "{}_symbscan.sqlite".format(datetime.now().strftime("%F"))
SQL_CONFIG_FILE = path.join(environ["HOME"], ".my.cnf")


def get_mysql_config(file):
    config = cp.ConfigParser()
    config.read(file)
    return {
        "host": config["client"]["host"],
        "port": int(config["client"]["port"]),
        "user": config["client"]["user"],
        "password": config["client"]["password"],
        "database": config["mysql"]["database"],
        "charset": "utf8mb4",
        "cursorclass": DictCursor,
    }


def get_sqlite_conn(file):
    sqlite_conn = sqlite3.connect(file)
    sqlite_conn.row_factory = sqlite3.Row
    return sqlite_conn


def get_mysql_conn(config_file):
    config = get_mysql_config(config_file)
    return pymysql.connect(**config)


def get_kml_poly(kml_file_name):
    with open(kml_file_name, "rb") as f:
        kml_file = pykml.parser.fromstring(f.read())

    kml_coords = str(kml_file.Document.Placemark.Polygon.outerBoundaryIs.LinearRing.coordinates).strip()
    kml_coords = [p for p in kml_coords.split(" ")]
    kml_coords = [(float(lng), float(lat)) for lng, lat, alt in [p.split(",") for p in kml_coords]]
    return kml_coords


def get_provinces(file):
    with open(file) as f:
        return [l.strip() for l in f.readlines() if l != ""]

    
def get_countries(file):
    with open(file) as f:
        return [l.strip() for l in f.readlines() if l != ""]

In [ ]:
KML_POLY = get_kml_poly(N_AMERICA_KML)
OMOCCURRENCES_LATS = [p[0] for p in KML_POLY]
OMOCCURRENCES_LNGS = [p[1] for p in KML_POLY]

OMOCCURRENCES_LATITUDE_RANGE = [min(OMOCCURRENCES_LATS), max(OMOCCURRENCES_LATS)]
OMOCCURRENCES_LONGITUDE_RANGE = [min(OMOCCURRENCES_LNGS), max(OMOCCURRENCES_LNGS)]

TARGET_FAMILIES = [
    'melittidae',
    'colletidae',
    'apidae',
    'megachilidae',
    'halictidae',
    'andrenidae'
]

TARGET_COUNTRIES = get_countries(COUNTRIES_FILE)
TARGET_PROVINCES = get_provinces(PROVINCES_FILE)

SCAN_QUERY = """
    SELECT * FROM omoccurrences
    WHERE (
        LOWER(family) IN ({})
        AND (
            (
                decimalLatitude BETWEEN {} AND {} 
                AND decimalLongitude BETWEEN {} AND {}
            )
            OR lower(country) in ({})
            OR lower(stateProvince) in ({})
        )
    )
""".format(
    ','.join(["'{}'".format(f) for f in TARGET_FAMILIES]),
    *OMOCCURRENCES_LATITUDE_RANGE,
    *OMOCCURRENCES_LONGITUDE_RANGE,
    ','.join(["'{}'".format(f) for f in TARGET_COUNTRIES]),
    ','.join(["'{}'".format(f) for f in TARGET_PROVINCES]),
)

#print(SCAN_QUERY)

In [29]:
CHUNK_SIZE=50000

if path.exists(SCAN_SQLITE):
    os_remove(SCAN_SQLITE)

scan_conn = get_mysql_conn(SQL_CONFIG_FILE)
sqlite_conn = get_sqlite_conn(SCAN_SQLITE)

try:
    with scan_conn:
        input_df = pd.read_sql(SCAN_QUERY, scan_conn, chunksize=CHUNK_SIZE)
        
        for chunk in input_df:
            chunk['source'] = 'scan'
            with sqlite_conn:
                chunk.to_sql(
                    "omoccurrences",
                    con=sqlite_conn,
                    index=False,
                    if_exists="append"
                )

except Exception as e:
    sqlite_conn.rollback()
    sqlite_conn.close()
    
    scan_conn.rollback()
    scan_conn.close()
    raise e

sqlite_conn.close()

In [30]:
%reset_selective -f ^chunk$
%reset_selective -f ^input_df$
%reset_selective -f ^sqlite_conn$
%reset_selective -f ^scan_conn$
gc_collect()

#%who

20

In [ ]:
gbif_df = pd.read_csv(GBIF_CSV, sep="\t", nrows=1)
gbif_cols = sorted(list(gbif_df.columns))

In [ ]:
sqlite_conn = get_sqlite_conn(SCAN_SQLITE)

try:
    with sqlite_conn:
        scan_cols_query = sqlite_conn.execute("select * from omoccurrences limit 1")
        scan_cols = sorted([desc[0] for desc in scan_cols_query.description])
                
except Exception as e:
    print(e, file=stderr)
    
finally:
    sqlite_conn.close()

In [ ]:
common_cols = list(np.intersect1d(gbif_cols, scan_cols))
#[print(c) for c in common_cols]

In [28]:
CHUNK_SIZE = 100000

sqlite_conn = get_sqlite_conn(SCAN_SQLITE)

try:   
    with sqlite_conn:
        gbif_df = pd.read_csv(GBIF_CSV, chunksize=CHUNK_SIZE, sep="\t", low_memory=False)
        for chunk in gbif_df:
            chunk['source'] = 'gbif'
            chunk = chunk[['source', *common_cols]]
            #display(chunk["scientificName"].head())
            #break
            
            chunk.to_sql(
                "omoccurrences",
                con=sqlite_conn,
                index=False,
                if_exists="append"
            )
            
except Exception as e:
    print(e, file=stderr)
    sqlite_conn.rollback()

finally:
    sqlite_conn.close()

In [ ]:
sqlite_conn = get_sqlite_conn(SCAN_SQLITE)

try:
    with sqlite_conn:
        # GBIF stores it as scientificName, SCAN stores it as sciName
        sqlite_conn.execute("update omoccurrences set scientificName = sciName where source = 'scan'")
        
        # Cleaning
        sqlite_conn.execute("update omoccurrences set scientificName = trim(scientificName)")
except Exception as e:
    print(e, file=stderr)
    sqlite_conn.rollback()
finally:
    sqlite_conn.close()


In [ ]:
def populate_sciname(row):
    rank_col = row["taxonRank"].lower()
    if rank_col in row:
        row["scientificName"] = row[rank_col]
    return row

def remove_parenthesis(row):
    sciName = row["scientificName"]
    
    if pd.isna(sciName):
        return row
    
    sciName = re.sub(r"\([^)]+\)", "", sciName)
    sciName = re.sub(r" +", " ", sciName).strip()
    row["scientificName"] = sciName
    return row

def remove_authorship(row):
    sciName = row["scientificName"]
    
    if pd.isna(sciName):
        return row
    
    sciName = re.sub(r" [A-Z][a-z]+,\s+\d{4}\s*$", "", sciName)
    sciName = re.sub(r" +", " ", sciName).strip()
    row["scientificName"] = sciName
    return row


In [ ]:
# ===== FILTERING =====

CHUNK_SIZE = 100000
N=10

sqlite_conn = get_sqlite_conn(SCAN_SQLITE)
    
try:
    with sqlite_conn:
        db_df = pd.read_sql("select occid, scientificName from omoccurrences", sqlite_conn, chunksize=CHUNK_SIZE)
        for chunk in db_df:
#             print("Original")
#             display(chunk.head(n=N))
            
            chunk = chunk.apply(remove_parenthesis, axis="columns")
            
#             print("Without parentheses")
#             display(chunk.head(n=N))
            
            chunk = chunk.apply(remove_authorship, axis="columns")
            
#             print("Without authorship")
#             display(chunk.head(n=N))
            
            chunk_valid_scinames = chunk[
                ~(pd.isna(chunk["scientificName"]) | (chunk["scientificName"] == ''))
            ]
            for _, row in chunk_valid_scinames.iterrows():
                sqlite_conn.execute(
                    "UPDATE omoccurrences SET scientificName = ? WHERE occid = ?",
                    (row["scientificName"], row["occid"])
                )

except Exception as e:
    sqlite_conn.rollback()
    sqlite_conn.close()
    raise e

sqlite_conn.close()